In [38]:
%matplotlib inline
import seaborn as sns, numpy as np, cPickle, pandas as pd, pylab,random,cPickle, matplotlib.pyplot as plt
from os import system,listdir
from scipy.cluster.hierarchy import fcluster,linkage,cophenet
from IPython.display import display, HTML
from os.path import join,dirname,basename,isdir
global familymatching_thresh 
from sklearn import decomposition,manifold,preprocessing,metrics
from sklearn.preprocessing import scale,normalize,StandardScaler
from scipy.stats import variation
import matplotlib.cm as cm

topdir = '/cluster/zeng/research/lineage'
familymatching_thresh = 0

## Barcode filtering and family analysis

In [73]:
def listdist(seqs1,seqs2,dist_thresh):
    ### If any element of seqs1 matches any element of seq2, we call it a match
    for idx1 in range(len(seqs1)):
        for idx2 in range(len(seqs2)):
            t_dist = dist(seqs1[idx1],seqs2[idx2])
            if t_dist <= dist_thresh:
                return 0
    return 1

def dist(seq1,seq2):
    ### Calculate the  Levenshtein distance distance of two string, considering 'N' as wildcard
    n1 = indexing(seq1,'N',False)
    n2 = indexing(seq2,'N',False)
    nindex = list(set.intersection(n1,n2))
    return sum([1 for x in nindex if seq1[x]!=seq2[x]])

def indexing(seq,key,equal):
    ### Find the element in the list that matches a key
    if equal:
        return set([i for i in xrange(len(seq)) if seq[i]==key])
    else:
        return set([i for i in xrange(len(seq)) if seq[i]!=key])

In [66]:
def analysis(mappingfile,family_analysis,barcode2filter,pe2readthresh):
    datacode = mappingfile.split('.raw.tsv')[0]
    
    ### Read the mapping result
    oridata = pd.read_csv(mappingfile,sep ='\t',dtype =\
                          {'barcodecnt':int,'barcode_moleculecnt':int,'readcnt':int,'moleculecnt':int})
    oridata['barcodes'] = oridata['barcodes'].apply(lambda x: np.asarray(x.split(',')))
    oridata['barcodereads_split'] = oridata['barcodereads_split'].apply(lambda x: np.asarray(map(int,x.split(','))))
    ori_barcode_coverage = pd.Series([ y for x in oridata['barcodes'] for y in x]).value_counts()
    ori_barcode_coverage.to_csv(datacode+'.bcCoverage.thresh'+str(pe2readthresh),sep='\t')
    
    ### For each cell, retain only the first two barcode with a threshold cutoff
    data = oridata.copy(deep=True)
    for index, row in data.iterrows():
        t_bc_read_cnt = row['barcodereads_split']
        topick = [x for x in np.argsort(t_bc_read_cnt)[-2:][::-1] if t_bc_read_cnt[x]>pe2readthresh]
        if len(topick) == 2:
            if t_bc_read_cnt[topick[1]] < t_bc_read_cnt[topick[0]]/2.0:
                topick = [topick[0]]
        if len(topick)>0:
            data.set_value(index,'barcodes',row['barcodes'][topick])
            data.set_value(index,'barcodereads_split',t_bc_read_cnt[topick])
            data.set_value(index,'barcode_moleculecnt',sum(t_bc_read_cnt[topick]))
            data.set_value(index,'barcodecnt',len(topick))
        else:
            data.set_value(index,'barcodes',['\\'])
    data['barcodes'] = data['barcodes'].apply(lambda x:','.join(x))
    
    ## How many cells does each barcode appear in?
    barcode_coverage = pd.Series([ y for x in data['barcodes'] for y in x.split(',')]).value_counts()
    barcode_coverage.to_csv(datacode+'.bcCoverage.postfilter.thresh'+str(pe2readthresh),sep='\t')
    bi_barcode_coverage = pd.Series([ ','.join(np.sort(x.split(','))) for x in data['barcodes'] ]).value_counts()
    bi_barcode_coverage.to_csv(datacode+'.bcCoverage.postfilter.bi.thresh'+str(pe2readthresh),sep='\t')
    
    ## Output the statstics
    stats= [['# of reads',oridata['readcnt'].sum()],\
            ['# of transcript molecules (unique UMI)',oridata['moleculecnt'].sum()],\
            ['# of transcript molecules (unique UMI) w. bc',oridata['barcode_moleculecnt'].sum()],\
            ['# of cells',len(oridata)],\
            ['# of barcodes',len(ori_barcode_coverage)],\
            ['# of cells after processing',len(data[data['barcodes']!= '\\'])],\
            ['# of barcodes after processing',len(barcode_coverage)],\
            ['# of barcodes after processing (concat two bc)',len(bi_barcode_coverage)],\
            ]
    summary = pd.DataFrame([[x[0],x[1]] for x in stats],columns=['Item','Num'])
    
    ### Family analysis
    family_summary = family(data[data['barcodes']!= '\\'],[datacode+'.familyanalysis-cellcentric'+\
                             '.thres'+str(pe2readthresh)+'.csv',datacode+'.familyanalysis-aggregated'+\
                             '.thres'+str(pe2readthresh)+'.csv']) if family_analysis else None
        
    return summary,family_summary

In [67]:
def reorder(x):
    t_barcodes = x['barcodes'].split(',')
    pick = np.argsort(t_barcodes)
    x['barcodes'] = ','.join([t_barcodes[y] for y in pick])
    x['barcodereads_split'] = [x['barcodereads_split'][y] for y in pick]
    return x

def family(data,outfiles):
    num_data = len(data)
    data.index = range(num_data)
    
    ## Clustering the cells by their lineage barcodes
    cond_dist = [listdist(data.get_value(idx1,'barcodes').split(','),data.get_value(idx2,'barcodes').split(','),\
                          familymatching_thresh) for idx1 in range(num_data-1) for idx2 in range(idx1+1,num_data)]
    link = linkage(cond_dist,method='single')
    
    ## Output cell-centric lineage information
    data['lineage'] = pd.Series(fcluster(link,0.5,criterion='distance')-1,index=data.index)
    data = data.sort_values(by='lineage').apply(lambda x: reorder(x),axis=1)
    data.to_csv(outfiles[0],index=False)
    
    ## Output lineage statistics aggregated cross cells
    lineage_info = []
    for name, group in data.groupby('lineage'):
        vc = group['barcodes'].value_counts()
        for idx,x in enumerate(vc):
            lineage_info.append([name,vc.index[idx],x])          
    pd.DataFrame(lineage_info,columns=['Lineage','Barcodes','Cnt']).to_csv(outfiles[1])
    
    ## Output the statistics
    stats= [['# of cells included in family analysis',num_data] ,\
            ['# of lineages',len(data['lineage'].unique())],\
            ['Size of lineages',list(data['lineage'].value_counts())]
             ]
    return pd.DataFrame([[x[0],x[1]] for x in stats],columns=['Item','Num'])

#### fth1 integrated barcodes

In [75]:
expts = ['mESC_fth1_time1_rep1','mESC_fth1_time1_rep2','mESC_fth1_time2_rep1','mESC_fth1_time2_rep2',\
         'mESC_fth1_timeboth_rep2']
expts_short = [x.replace('time','T').replace('rep','R') for x in expts]
for t_thresh in [1,3,10]:
    allsummary = None
    allsummary_family = None
    for idx, expt in enumerate(expts):
        mappingfile = join(topdir,expt,'post_split/cell-lineage_mapping_quality20_mismatch2_prefixsuffixtol2.raw.tsv')
        t_s,t_family = analysis(mappingfile,True,[],t_thresh)
        t_s.columns = ['Item',expts_short[idx]]
        t_family.columns = ['Item',expts_short[idx]]
        allsummary = t_s if allsummary is None else allsummary.merge(t_s,on='Item')
        allsummary_family = t_family if allsummary_family is None else allsummary_family.merge(t_family,on='Item')
    combined = allsummary.append(allsummary_family)
    combined['Thresh'] = pd.Series(t_thresh,index=combined.index)
    display(combined)
    combined.to_csv(join(topdir,'fth1.stats.thresh'+str(t_thresh)+'.csv'),index=False)

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,mESC_fth1_Tboth_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,18220548,1
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,509308,1
2,# of transcript molecules (unique UMI) w. bc,1914,72559,36020,114088,140148,1
3,# of cells,196,218,144,232,324,1
4,# of barcodes,16,385,118,308,522,1
5,# of cells after processing,122,218,144,232,324,1
6,# of barcodes after processing,4,13,7,12,13,1
7,# of barcodes after processing (concat two bc),4,17,7,17,20,1
0,# of cells included in family analysis,122,218,144,232,324,1
1,# of lineages,3,9,4,6,7,1


,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,mESC_fth1_Tboth_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,18220548,3
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,509308,3
2,# of transcript molecules (unique UMI) w. bc,1914,72559,36020,114088,140148,3
3,# of cells,196,218,144,232,324,3
4,# of barcodes,16,385,118,308,522,3
5,# of cells after processing,57,217,143,232,324,3
6,# of barcodes after processing,2,14,7,12,13,3
7,# of barcodes after processing (concat two bc),2,18,7,17,20,3
0,# of cells included in family analysis,57,217,143,232,324,3
1,# of lineages,1,9,4,6,7,3


,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,mESC_fth1_Tboth_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,18220548,10
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,509308,10
2,# of transcript molecules (unique UMI) w. bc,1914,72559,36020,114088,140148,10
3,# of cells,196,218,144,232,324,10
4,# of barcodes,16,385,118,308,522,10
5,# of cells after processing,8,217,143,232,324,10
6,# of barcodes after processing,2,14,7,12,13,10
7,# of barcodes after processing (concat two bc),2,18,7,17,20,10
0,# of cells included in family analysis,8,217,143,232,324,10
1,# of lineages,1,9,4,6,7,10


#### fth1 barcodelets

In [81]:
expts = ['mESC_fth1_time1_rep1','mESC_fth1_time1_rep2','mESC_fth1_time2_rep1','mESC_fth1_time2_rep2']
expts_short = [x.replace('time','T').replace('rep','R') for x in expts]
for t_thresh in [1,3,10]:
    allsummary = None
    allsummary_family = None
    for t_idx,expt in enumerate(expts):
        mappingfile = join(topdir,expt,'post_split',\
                           'cell-lineage_mapping_quality20_mismatch2_prefixsuffixtol2.barcodelet.raw.tsv')
        t_s,t_family = analysis(mappingfile,False,[],t_thresh)
        t_s.columns = ['Item',expts_short[t_idx]]
        allsummary = t_s if allsummary is None else allsummary.merge(t_s,on='Item')
    allsummary['Thresh'] = pd.Series(t_thresh,index=allsummary.index)
    display(allsummary)
    allsummary.to_csv(join(topdir,'barcodelet.stats.thresh'+str(t_thresh)+'.csv'),index=False)

,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,1
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,1
2,# of transcript molecules (unique UMI) w. bc,296,103,228,115,1
3,# of cells,196,218,144,232,1
4,# of barcodes,77,40,67,60,1
5,# of cells after processing,63,18,37,13,1
6,# of barcodes after processing,25,12,24,6,1
7,# of barcodes after processing (concat two bc),27,11,23,6,1


,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,3
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,3
2,# of transcript molecules (unique UMI) w. bc,296,103,228,115,3
3,# of cells,196,218,144,232,3
4,# of barcodes,77,40,67,60,3
5,# of cells after processing,12,1,14,0,3
6,# of barcodes after processing,8,2,5,1,3
7,# of barcodes after processing (concat two bc),8,2,5,1,3


,Item,mESC_fth1_T1_R1,mESC_fth1_T1_R2,mESC_fth1_T2_R1,mESC_fth1_T2_R2,Thresh
0,# of reads,2998912,10967607,6146482,8027986,10
1,# of transcript molecules (unique UMI),173648,128075,135995,173619,10
2,# of transcript molecules (unique UMI) w. bc,296,103,228,115,10
3,# of cells,196,218,144,232,10
4,# of barcodes,77,40,67,60,10
5,# of cells after processing,0,0,0,0,10
6,# of barcodes after processing,1,1,1,1,10
7,# of barcodes after processing (concat two bc),1,1,1,1,10
